#### Imports

In [0]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import PIL.ImageOps    
from PIL import Image
import numpy as np
import itertools
import urllib
import math
import sys
import os
from google.colab import drive
from scipy.spatial.distance import squareform

#### Helper functions

In [0]:
# Get node positions using palslayout

file = urllib.request.urlretrieve('https://github.com/kalngyk/repoman/raw/master/palslayout.so', 'palslayout.so')
import palslayout

def getpos(G, communities):
    node2idx = dict([(x, i) for (i, x) in enumerate(G.nodes)])
    clusters = []
    for n, c in enumerate(communities):
        indices = [node2idx[x] for x in G.nodes if x in c]
        for i in indices:
            clusters.append([i, n])
    clusters = np.array(clusters)
    edges = np.array([[node2idx[x], node2idx[y]] for x, y in G.edges])
    pos = palslayout.getpos(edges, clusters)
    nodes = np.array(G.nodes)
    results = dict([(nodes[int(i)], np.array([j, k])) for i, j, k in pos])
    return results

#### Give access to Google Drive

In [3]:
savedir = ""
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    savedir = "/content/gdrive/My Drive/colab/"
    try:
        os.mkdir(savedir)
    except FileExistsError:
        print("Directory \"%s\" already exists" % (savedir))

print("Content of {}:".format(savedir))
os.listdir(savedir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Directory "/content/gdrive/My Drive/colab/" already exists
Content of /content/gdrive/My Drive/colab/:


['labels.txt',
 'dissim.tsv',
 'gephi-edges.csv',
 'pals-edges.txt',
 'pals-community.txt',
 'test.tsv']

In [9]:
import os
path = "/content/gdrive/My Drive/CD_Algorithms/pkl"
pkllist = os.listdir(path)
pkllist.remove('.gitignore')
pkllist

['hot_project_80_norm_LP.pkl',
 'hot_project_80_norm_GN.pkl',
 'hot_project_80_norm_MC.pkl',
 'hot_project_100_norm_GN.pkl',
 'hot_project_100_norm_MC.pkl',
 'hot_project_100_norm_LP.pkl']

In [11]:
import seaborn as sns
import pickle

for pklfile in pkllist:
    print(pklfile)
    filename = path + "/" + pklfile
    with open(filename, 'rb') as f:
        G, communities = pickle.load(f)
    
    # Find nodes layout using pals
    pos = getpos(G, communities)
    
    # Gives colors to the clusters
    color_map = []
    cls_elem_count = np.array([len(c) for c in communities])
    n2c = dict()
    for i, c in enumerate(communities):
        for n in c:
            n2c[str(n)] = i
    palette = sns.color_palette(None, len(cls_elem_count[cls_elem_count > 1]))
    for node in G.nodes:
        c = n2c[node]
        if cls_elem_count[c] == 1:
            idx = 0
        else:
            idx = c % len(palette)
        color_map.append(palette[idx])

    plt.figure(figsize=(17, 15))
    nx.draw(G, pos=pos, node_color=color_map, edge_color='grey', with_labels=True)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.